In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torchvision import transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
            super(Net, self).__init__()
            self.layer1 = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
            self.layer2 = nn.Sequential(
                nn.Conv2d(32, 64, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
            self.layer3 = nn.Sequential(
                nn.Conv2d(64, 128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
            self.drop = nn.Dropout()
            self.fc1 = nn.Linear(32*52*52, 1024)
            self.fc2 = nn.Linear(1024, 256)
            self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

model = Net()
print(model)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device=device)

In [ ]:
dataset = pd.read_csv('driver_imgs_list.csv')
import os
from IPython.display import display, Image
import matplotlib.image as mpimg

activity_map = {'c0': 'Safe driving',
                'c1': 'Texting - right',
                'c2': 'Talking on the phone - right',
                'c3': 'Texting - left',
                'c4': 'Talking on the phone - left',
                'c5': 'Operating the radio',
                'c6': 'Drinking',
                'c7': 'Reaching behind',
                'c8': 'Hair and makeup',
                'c9': 'Talking to passenger'}

plt.figure(figsize = (12, 20))
image_count = 1
BASE_URL = 'imgs/train/'
for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(5, 2, image_count)
                image_count += 1
                image = mpimg.imread(BASE_URL + directory + '/' + file)
                plt.imshow(image)
                plt.title(activity_map[directory])

In [ ]:
import train_loader
from torch.utils.data import DataLoader

train_data = train_loader.TrainDataset('imgs/train/', train=True)
train_dataloader = DataLoader(dataset=train_data,shuffle=True, batch_size=32, num_workers=4)

vali_data = train_loader.TrainDataset('imgs/train/', train=False)
vali_dataloader = DataLoader(dataset=vali_data, shuffle=False, batch_size=32, num_workers=4)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)



In [ ]:
def val(model, dataloader):
    model.eval()
    correct, total, test_loss = 0, 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)

            # Checking answer
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            test_loss += criterion(output, labels).item()

    model.train()
    print('Images Tested =', total)
    print('Accuracy = {:.2f}%\n'.format(100. * correct / total))
    return 100. * correct / total

In [ ]:
import time
num_epochs = 100


for epoch in range(1, num_epochs + 1):
        st = time.time()
        running_loss = 0.0
        for idx, (images, labels) in enumerate(train_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            # Clear gradients
            optimizer.zero_grad()
            # Get loss
            output = model(images)
            loss = criterion(output, labels)
            # Forward, back, optimize
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # append data every 10 mini-batches, print every 100 mini-batches
            if idx % 100 == 0:
                print('Epoch {} -> [{}/{} ({:.0f}%)] Loss: {:.6}'.format(
                    epoch, idx * len(images.to(device)),
                    len(train_dataloader.dataset),
                    100. * idx / len(train_dataloader),
                    running_loss / len(train_dataloader)))
                running_loss = 0.0

        else:
            print("epoch time : %f s" % (time.time()-st))
            accuracy = val(model, vali_dataloader)
            trained_model_path = './trained_models/watchdog/'
            if epoch % 5 == 0:
                if not os.path.isdir(trained_model_path):
                    os.makedirs(trained_model_path)
                torch.save(model, trained_model_path + '%d'%epoch + '_' + '%f'%accuracy + '.pkl')


In [ ]:
torch.save(model.state_dict(), './model.pth')
